In [1]:
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import folium
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import getName
from ipynb.fs.defs.functions import translate_name
from ipynb.fs.defs.functions import get_row
from ipynb.fs.defs.functions import get_column

dft = pd.read_csv("../distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

## Interpretation of biblio.sol

In [2]:
def parseEl(x):
    temp = x[x.find("(")+1:x.find(")")]
    temp = temp.split(",")
    return temp[0], temp[1]

indx = [getName(dft, x) for x in dft.columns]
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]
biblio = pd.read_csv("../turni_f/fine_optimized_start/biblio.sol", delimiter=" ", encoding = "ISO-8859-1", names = ['x','b'])
x = pd.DataFrame({}, columns=["city", "day"])
biblio = biblio[biblio["b"]==1]
for el in biblio["x"]:
    s, d = parseEl(el)
    if(int(s)>=15):
        x = x.append({"city" : indx[int(s)+1], "day":week[int(d)]}, ignore_index = True)
    else:
        x = x.append({"city" : indx[int(s)], "day":week[int(d)]}, ignore_index = True)
        
lenght = max([len(x[x["day"]==d]) for d in week])
optimized_start_schedule = pd.DataFrame({})
optimized_start_schedule["pad"] = pd.Series([np.nan]*lenght)

for d in week:
    selection = x["city"][x["day"]==d].reset_index()["city"]
    optimized_start_schedule = optimized_start_schedule.join(
                                        pd.DataFrame(x["city"][x["day"]==d].reset_index()["city"].rename(d)))
optimized_start_schedule = optimized_start_schedule[week]
optimized_start_schedule

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Agnadello, Marconi, 6","Capralba, Piave, 2","Bonemerse, Roma, 25","Annicco, Giuseppe Garibaldi, 6","Casalmaggiore, Guglielmo Marconi, 8"
1,"Bagnolo Cremasco, Roma, 3","Casale Cremasco-Vidolasco, Roma, 56","Casalmaggiore, Guglielmo Marconi, 8","Camisano, Trieste, 57","Cingia de' Botti, Giuseppina, 79"
2,"Capergnanica, XI febbraio, 7","Castelleone, Roma, 67","Casalpusterlengo, Pusterla 7","Casalbuttano ed Uniti, Jacini, 23","Crema, Civerchi, 9"
3,"Crema, Civerchi, 9","Grumello Cremonese ed Uniti, Roma, 2","Crema, Civerchi, 9","Casalpusterlengo, Pusterla 7","Cremona, Ugolani Dati, 4"
4,"Dovera, Oldrini 2","Pizzighettone, Giuseppe Garibaldi, 18","Cremona, Ugolani Dati, 4","Castelleone, Roma, 67","Gadesco-Pieve Delmona, Giovanni Lonati 9"
5,"Izano, Gerardo da Izano, 4/C","Sergnano, IV Novembre, 10","Gadesco-Pieve Delmona, Giovanni Lonati 9","Castelverde, Ubaldo Ferrari, 15","Grontardo, Roma, 16"
6,"Spino d'Adda, Circonvallazione, 13","Trigolo, Roma, 26","Gussola, Comaschi, 1","Crema, Civerchi, 9","Isola Dovarese, Giacomo Matteotti, 1"
7,"Trescore Cremasco, Vittorio Veneto, 1",NaN,"Monte Cremasco, Roma, 12","Cremona, Ugolani Dati, 4","Martignana di Po, Libertà, 64"
8,"Vaiano Cremasco, Gloriosi Caduti, 3",NaN,"Montodine, XXV Aprile, 10","Fiesco, Giuseppe Parini, 10","Offanengo, Clavelli Martini, 5"
9,"Casaletto Vaprio, Maggiore, 7",NaN,"Offanengo, Clavelli Martini, 5","Madignano, Portici, 4","Pandino, Castello, 15"


In [3]:
optimized_start_schedule.to_csv("../turni_f/fine_optimized_start/foss_start.csv", sep=";", encoding = "ISO-8859-1", index = False)

## Extract f1 and f2 optimized start schedule

In [5]:
total = pd.read_csv("../turni_f/fine_optimized_start/foss_start.csv", delimiter=";", encoding = "ISO-8859-1")
df1 = pd.DataFrame({})
df1["pad"] = pd.Series([np.nan]*len(total))
df2 = pd.DataFrame({})
df2["pad"] = pd.Series([np.nan]*len(total))
week = ["lunedi", "martedi", "mercoledi", "giovedi", "venerdi"]

for day in week:
    if day in ["lunedi", "venerdi"]:
        df1 = df1.join(total[day])
    else:
        start = dft[get_row(dft, 'Cremona, sesto 39').values]
        f1 = []
        f2 = []
        row1 = start
        row2 = start
        for to_insert in total[day].dropna():
            value1=row1.loc[:,get_column(dft, to_insert).values]
            value2=row2.loc[:,get_column(dft, to_insert).values]
            if(float(value1.values[0])<=float(value2.values[0])):
                f1.append(to_insert)
                row1 = dft[get_row(dft, to_insert).values]
            else:
                f2.append(to_insert)
                row2 = dft[get_row(dft, to_insert).values]
                           
        df1 = df1.join(pd.Series(f1, dtype = str).rename(day))
        df2 = df2.join(pd.Series(f2, dtype = str).rename(day))

df1 = df1[week]
df2 = df2[["martedi", "mercoledi", "giovedi"]]

In [6]:
df1.to_csv("../turni_f/fine_optimized_start/foss_f1.csv", sep=";", encoding = "ISO-8859-1", index = False)
df2.to_csv("../turni_f/fine_optimized_start/foss_f2.csv", sep=";", encoding = "ISO-8859-1", index = False)